This code must be run locally, never on Google Colab, because there are functions that don't work properly at this platform.

In [ ]:
# When running it, change the directoy to that where this file is in.
directory  = "/home/merlin/coding/ic/createDataset"

In [ ]:
# Google Drive folder IDs:
dataID =
dataWithResults =
dataOnDataset =

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.files import GoogleDriveFile

import numpy as np
import pandas as pd

import os
import cv2

from Writer import Writer

In [ ]:
# Verifying Google Drive API authorization. For this is necessary a Google account. To avoid using personal accounts,
# there is a specific account to this, that should be logged in, when asked, to script work properly.
# Email:
# Password:

gAuth = GoogleAuth()
gAuth.LocalWebserverAuth()
gDrive = GoogleDrive(gAuth)

In [ ]:
def downloadExamResults():
    '''
    Access the project Google Drive and download the CSV containg the exam
    results of all pacients.
    '''

    # Get a list of files in the 'dataID' folder.
    files = gDrive.ListFile({'q':  "\'" + dataID + "\' in parents and trashed=false" }).GetList()

    # Searching and downloading the exam results CSV.
    for f in files:
        if f['title'] == "examResults.csv":
            f.GetContentFile("examResults.csv")

def searchDatasetCsv() -> bool:
    '''
    Access the project Google Drive. If the dataset already exists, downloads it
    and returns True, else returns False.
    '''

    # Get a list of files in the 'dataID' folder.
    files = gDrive.ListFile({'q':  "\'" + dataID + "\' in parents and trashed=false" }).GetList()

    # Searching and downloading the dataset CSV.
    for f in files:
        if f['title'] == "dataset.csv":
            f.GetContentFile("dataset.csv")
            return True

    return False

def downloadVideo(pacientId: str):
    '''
    Access the folder of a certain pacient in the project Google Drive and
    downloads its video.
    '''

    # Get a list of files in the pacient folder.
    files = gDrive.ListFile({'q':  "\'" + pacientId + "\' in parents and trashed=false" }).GetList()

    for f in files:
        if f['title'] == "video_dedo.mp4":
            f.GetContentFile("video_dedo.mp4")

def extractResultsFromCsv(pacientId: str) -> pd.Series:
    '''
    Opens the exam results CSV and returns the results of the 'pacientId'.
    '''

    # Opening the CSV file.
    f = pd.read_csv("examResults.csv")

    # Searching for the pacient results.
    idList = f['ID'].tolist()
    index = idList.index(pacientId)

    # Returning the results of the given pacient.
    return f.iloc[index][1:]

def extractTsFromVideo() -> list:
    '''
    Opens a video and returns a list containing the RGB time series.
    '''

    # Opening video.
    video = cv2.VideoCapture("video_dedo.mp4")

    # Getting video FPS.
    fps = video.get(cv2.CAP_PROP_FPS)

    # First frame to be processed is on 3s, then we skip 2s of frames.
    frameId = fps * 2

    # Initializing arrays that will store the mean of each frame pixels.
    redMeans = np.array([])
    greenMeans = np.array([])
    blueMeans = np.array([])

    # Going through video and diving in frames. We collect only 900 observations.
    for i in range(900):
        video.set(cv2.CAP_PROP_POS_FRAMES, frameId)
        ret, frame = video.read()
        frameId += 1

        # If the frame exists...
        if(ret == True):
            # Diving the frame on its Red, Green and Blue channels.
            blue,green,red = cv2.split(frame)

            # Transforming the 2D frame in an array.
            pixelsRed = red.flatten()
            pixelsGreen = green.flatten()
            pixelsBlue = blue.flatten()

            # Calculating mean.
            redFrameMean = np.mean(pixelsRed)
            greenFrameMean = np.mean(pixelsGreen)
            blueFrameMean = np.mean(pixelsBlue)

            # Adding the means in the properly array.
            redMeans = np.append(redMeans, redFrameMean)
            greenMeans = np.append(greenMeans, greenFrameMean)
            blueMeans = np.append(blueMeans, blueFrameMean)

    return [[redMeans, greenMeans, blueMeans]]

def writeInfo(timeSeries: np.ndarray, data: list, header: list, writer: Writer, flag: bool, i: int):
    '''
    Writes the pacient information on the dataset file using the proper function.
    '''

    if flag == False:
        if i == 0:
            writer.writeFile(timeSeries, pd.DataFrame([data], columns=header))
        else:
            if i == 1:
                writer.setFile('dataset.csv', 'old')

            writer.writeOnePerson(timeSeries, pd.DataFrame([data], columns=header))
    else:
        writer.writeOnePerson(timeSeries, pd.DataFrame([data], columns=header))

def movePacientFolder(pacient: GoogleDriveFile):
    '''
    Transfers a pacient folder from 'Dados com Resultado' to 'Dados no Dataset'.
    '''

    pacient['parents'] = [{'id': dataOnDataset}]
    pacient.Upload()

def uploadDataset():
    '''
    Uploads the dataset created on project Google Drive.
    '''

    # Get a list of files in the 'dataID' folder.
    files = gDrive.Lists({'q':  "\'" + dataID + "\' in parents and trashed=false" }).GetList()

    driveFile = None

    # Searching for an old dataset file.
    for f in files:
        if f['title'] == "dataset.csv":
            driveFile = f
            break

    if driveFile == None:
        driveFile = gDrive.CreateFile({
            'parents': [{'id': dataID}],
            'title': "dataset.csv"
        })
    driveFile.SetContentFile("dataset.csv")
    driveFile.Upload()

In [ ]:
# Downloading the file with laboratorial exam results.
downloadExamResults()

datasetFlag = searchDatasetCsv()

# Get a list of pacients that already have their exam results.
pacients = gDrive.ListFile({'q':  "\'" + dataWithResults + "\' in parents and trashed=false" }).GetList()

#  Initializing the writer.
writer = Writer()
if datasetFlag == False:
    writer.setFile('dataset.csv', 'new')
else:
    writer.setFile('dataset.csv', 'old')

# Iterating through pacients, getting their exam results and RGB time series,
# and writing these information properly on the dataset file.
for i, pac in enumerate(pacients):
    print(str(i) + " ---> " + pac['title'])

    downloadVideo(pac['id'])

    examResults = extractResultsFromCsv(pac['title'])
    timeSeries = extractTsFromVideo()
    infoSexSkin = pac['title']

    examResults = examResults.dropna()
    data = [infoSexSkin[1]] + [infoSexSkin[0]] + (examResults.values).tolist()
    header = ['SEXO', 'SKINCOLOR'] + examResults.index.to_list()

    writeInfo(timeSeries, data, header, writer, datasetFlag, i)
    movePacientFolder(pac)
    os.remove("video_dedo.mp4")

uploadDataset()
os.remove("examResults.csv")
os.remove("dataset.csv")